In [9]:
import pyrealsense2 as rs
import numpy as np

In [10]:
pipeline = rs.pipeline()

In [11]:
config = rs.config()
config.enable_stream(rs.stream.color, 0, 0, rs.format.rgb8, 30)
pipeline.start(config)

In [12]:
profile = pipeline.get_active_profile()
intrinsics = (
    profile.get_stream(rs.stream.color).as_video_stream_profile().get_intrinsics()
)

In [13]:
fx = intrinsics.fx  # Focal length in x-direction
fy = intrinsics.fy  # Focal length in y-direction
cx = intrinsics.ppx  # Principal point x-coordinate
cy = intrinsics.ppy  # Principal point y-coordinate

In [14]:
pipeline.stop()

In [15]:
print("fx: ", fx)
print("fy: ", fy)
print("cx: ", cx)
print("cy: ", cy)

fx:  912.3776245117188
fy:  911.2467651367188
cx:  637.6239013671875
cy:  359.3946533203125


In [ ]:
fx = 912.3776245117188
fy = 911.2467651367188
cx = 637.6239013671875
cy = 359.3946533203125

In [47]:
# get transformation matrix for depth map to xyz coordinates

transformation_matrix = np.array(
    [
        [fx, 0, cx],
        [0, fy, cy],
        [0, 0, 1],
    ]
)

In [32]:
import cv2
import numpy as np


def rectify_image(
    source_image, source_intrinsics, target_intrinsics, transformation_matrix
):
    # Load the source image
    img = cv2.imread(source_image)

    # Retrieve the source image's camera intrinsics
    fx, fy = source_intrinsics["fx"], source_intrinsics["fy"]
    cx, cy = source_intrinsics["cx"], source_intrinsics["cy"]
    # distortion_coeffs = source_intrinsics['distortion_coeffs']

    # Compute the transformed camera intrinsics
    transformed_fx = transformation_matrix[0, 0] * fx + transformation_matrix[0, 2] * cx
    transformed_fy = transformation_matrix[1, 1] * fy + transformation_matrix[1, 2] * cy
    transformed_cx = transformation_matrix[0, 2] * fx + transformation_matrix[0, 2] * cx
    transformed_cy = transformation_matrix[1, 2] * fy + transformation_matrix[1, 2] * cy

    transformed_intrinsics = {
        "fx": transformed_fx,
        "fy": transformed_fy,
        "cx": transformed_cx,
        "cy": transformed_cy,
        #'distortion_coeffs': distortion_coeffs
    }

    print("Transformed camera intrinsics: ", transformed_intrinsics)

    # Rectify the source image
    mapx, mapy = cv2.initUndistortRectifyMap(
        cameraMatrix=np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]]),
        distCoeffs=None,
        R=transformation_matrix[:, :3],
        newCameraMatrix=np.array(
            [
                [transformed_fx, 0, transformed_cx],
                [0, transformed_fy, transformed_cy],
                [0, 0, 1],
            ]
        ),
        size=(img.shape[1], img.shape[0]),
        m1type=cv2.CV_32FC1,
    )

    rectified_img = cv2.remap(img, mapx, mapy, cv2.INTER_LINEAR)

    return rectified_img, transformed_intrinsics


# Example usage

# Source image path
source_image = "../data/processed/" + sample + "/Input_RGBImage.png"

# Source image camera intrinsics
source_intrinsics = {
    "fx": 500.0,
    "fy": 500.0,
    "cx": 320.0,
    "cy": 240.0,
    #'distortion_coeffs': [1.0, 0.0, 0.0, 0.0, 0.0]
}

# Target image camera intrinsics
target_intrinsics = {
    "fx": 500.0,
    "fy": 500.0,
    "cx": 320.0,
    "cy": 240.0,
    #'distortion_coeffs': [1.0, 0.0, 0.0, 0.0, 0.0]
}

# Transformation matrix representing the mapping between source and target intrinsics
transformation_matrix = np.array(
    [
        [target_intrinsics["fx"] / source_intrinsics["fx"], 0, 0],
        [0, target_intrinsics["fy"] / source_intrinsics["fy"], 0],
        [0, 0, 1],
    ]
)

# Rectify the image using camera intrinsics
rectified_image, rectified_intrinsics = rectify_image(
    source_image, source_intrinsics, target_intrinsics, transformation_matrix
)

# print rectified camera intrinsics
print("Rectified camera intrinsics: ", rectified_intrinsics)

# print shape of rectified image
print("Rectified image shape: ", rectified_image.shape)
# save rectified image
cv2.imwrite("../output/Input_RGBImage_rectified.png", rectified_image)

# Display the rectified image
cv2.imshow("Rectified Image", rectified_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Transformed camera intrinsics:  {'fx': 820.0, 'fy': 740.0, 'cx': 820.0, 'cy': 740.0}
Rectified camera intrinsics:  {'fx': 820.0, 'fy': 740.0, 'cx': 820.0, 'cy': 740.0}
Rectified image shape:  (480, 640, 3)


In [36]:
import cv2
import numpy as np


def align_intrinsics(input_image, input_intrinsics, desired_intrinsics):
    # Load the input image
    image = cv2.imread(input_image)

    # Extract the input intrinsics
    fx, fy = input_intrinsics["fx"], input_intrinsics["fy"]
    cx, cy = input_intrinsics["cx"], input_intrinsics["cy"]

    # Extract the desired intrinsics
    desired_fx, desired_fy = desired_intrinsics["fx"], desired_intrinsics["fy"]
    desired_cx, desired_cy = desired_intrinsics["cx"], desired_intrinsics["cy"]

    # Compute the scale factors
    scale_x = desired_fx / fx
    scale_y = desired_fy / fy

    # Compute the translation offsets
    tx = (desired_cx - cx) * scale_x
    ty = (desired_cy - cy) * scale_y

    # Generate the transformation matrix
    transformation_matrix = np.array([[scale_x, 0, tx], [0, scale_y, ty], [0, 0, 1]])

    # Apply the transformation to the image
    aligned_image = cv2.warpPerspective(
        image, transformation_matrix, (image.shape[1], image.shape[0])
    )

    return aligned_image


# Example usage
input_image_path = "../data/processed/" + sample + "/Input_RGBImage.png"
input_intrinsics = {"fx": 912.37, "fy": 911, "cx": 637, "cy": 359}
desired_intrinsics = {"fx": 914, "fy": 1000, "cx": 640, "cy": 361}

aligned_image = align_intrinsics(input_image_path, input_intrinsics, desired_intrinsics)

# Display the aligned image
cv2.imshow("Aligned Image", aligned_image)
cv2.waitKey(0)
cv2.destroyAllWindows()